In [21]:
stopwords = list(set([
    '이', '가', '은', '는', '을', '를', '의', '에', '에서', '에게', '께', '로', '으로', '하다', '있다',
    '와', '과', '보다', '처럼', '만큼', '같이', '까지', '마저', '조차', '부터', 
    '이나', '나', '이며', '며', '등', '하다', '한다', '하고', '하니', '하면', 
    '되어', '되다', '되고', '되니', '입니다', '습니다', 'ㅂ니다', '어요', '아요', '다', '방이', '제대로',
    '고', '면', '게', '지', '죠',
    '그리고', '그러나', '하지만', '그런데', '그래서', '그러면', '그러므로', '따라서', 
    '또한', '또는', '및', '즉', '한편', '반면에', '근데',
    '나', '저', '우리', '저희', '너', '너희', '당신', '그', '그녀', '그들', '누구', '그렇다',
    '무엇', '어디', '언제', '어느', '이것', '그것', '저것', '여기', '거기', '저기', 
    '이쪽', '그쪽', '저쪽',
    '하나', '둘', '셋', '넷', '다섯', '여섯', '일곱', '여덟', '아홉', '열',
    '일', '이', '삼', '사', '오', '육', '칠', '팔', '구', '십', '백', '천', '만',
    '첫째', '둘째', '셋째',
    '바로', '때', '것', '수', '문제', '경우', '부분', '이다',
    '내용', '결과', '자체', '가지', '있다',
    '않았어요', '있었어요', '했어요', '했는데요', '있는데요', '합니다', '없다', '나다','생각하다',
    '했다', '같다', '네요','아니다',
    '좀', '너무', '정말', '많이', '조금',
    '사장', '이용', '용하다', '물이', '매우',
    '뿐', '대로', '만', '따름', '나름', '김에', '터',
    '아', '아이고', '아이구', '아하', '어', '그래', '응', '네', '예', '아니', '않다', '안되다','안','그냥',
    '가다', '오다', '주다', '말다', '나다', '받다', '알다', '모르다', '싶다', '생각하다', '들다'
]))

In [22]:
import pandas as pd

df = pd.read_csv('keyword_test.csv', encoding='utf-8-sig')

# 키워드 불러오기 (이미 저장된 파일 사용)
keyword_df = pd.read_csv("숙소_관련_키워드.csv", encoding='utf-8-sig')
keyword_df['keyword'] = keyword_df['keyword'].str.strip()

In [23]:
from konlpy.tag import Okt
import pandas as pd
from tqdm import tqdm

# 형태소 분석기
okt = Okt()


In [24]:
# 사전 단어들도 형태소 분석 후 정규화 (옵션)
def normalize_keywords(keywords):
    normalized = set()
    for kw in keywords:
        tokens = okt.pos(kw, stem=True)
        for word, pos in tokens:
            if pos in ['Noun', 'Adjective', 'Verb']:
                if word not in stopwords :
                    normalized.add(word)
    return normalized

keyword_set = normalize_keywords(keyword_df['keyword'])

In [ ]:
keyword_set

: 

In [25]:

# 숙소 키워드만 추출하는 함수
def extract_keywords(text):
    try:
        tokens = okt.pos(str(text), stem=True)
        return " ".join([
            word for word, pos in tokens
            if pos in ['Noun', 'Adjective', 'Verb'] and word in keyword_set
        ])
    except:
        return ""

# 적용 (진행률 표시)
tqdm.pandas(desc="숙소 키워드 추출 중")
df['keywords'] = df['clean_reviews'].progress_apply(extract_keywords)

숙소 키워드 추출 중: 100%|██████████| 171762/171762 [17:16<00:00, 165.67it/s]


In [26]:
df = df[['review_id','content','clean_reviews','keywords']]

In [27]:
df.to_csv('keyword_test_result.csv', encoding='utf-8-sig', index=False)